In [2]:
import numpy as np
import cv2
import os
import random
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,Activation
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [3]:



def traiterImage (image_path):
  # Charger l'image et la redimensionner
  image = Image.open(image_path)
  image = image.resize((100, 100))

  image_array = np.array(image)

  # Vérifier la forme
  print(image_array.shape)

  return image_array/255



def transformationImageEnTableau(principale,sousDossier,data):


  for categories in sousDossier:
      folder=os.path.join(principale,categories)
      print(folder)
      label=sousDossier.index(categories)


      for img in os.listdir(folder):
          img=os.path.join(folder,img)
          img_arr=cv2.imread(img)
          if img_arr is not None:  # Check if the image is successfully loaded
              img_arr = cv2.resize(img_arr, (100, 100))
              data.append([img_arr, label])
          else:
              print(f"Failed to load image {img}")
  return data






def find_Class(directory_path):
    if not os.path.exists(directory_path):
        raise ValueError(f"The directory '{directory_path}' does not exist.")
    if not os.path.isdir(directory_path):
        raise ValueError(f"The path '{directory_path}' is not a directory.")

    # Get a list of all entries in the directory
    all_entries = os.listdir(directory_path)

    # Filter out only directories
    folders = [entry for entry in all_entries if os.path.isdir(os.path.join(directory_path, entry))]

    return folders

In [4]:
from google.colab import drive
drive.mount('/ressources')



Mounted at /ressources


In [5]:
#je liste tous mes sous dossier du repertoir train :entrainement et test respectivement dans sous_dossier_train et sous_dossier_test

principale_train= r"/ressources/My Drive/ressources/BDF/DATASET/train"
sous_dossier_train= []
sous_dossier_train = find_Class(principale_train)

principale_test= r"/ressources/My Drive/ressources/BDF/DATASET/test"
sous_dossier_test= []
sous_dossier_test = find_Class(principale_test)


print(f"{principale_train} {sous_dossier_train}")


print(f"{principale_test} {sous_dossier_test}")








/ressources/My Drive/ressources/BDF/DATASET/train ['1', '2', '3', '7', '6', '4', '5']
/ressources/My Drive/ressources/BDF/DATASET/test ['3', '7', '2', '4', '6', '1', '5']


In [6]:
#convertion des images en liste pour la manipulation ici nous mettons tout dans la variable data

emotions={"1":"surprise", "2":"fear", "6":"angry", "7":"neutral", "5":"sad", "3":"disgust", "4":"happy"}
data=[]
data= transformationImageEnTableau(principale_train,sous_dossier_train,data)

data= transformationImageEnTableau(principale_test,sous_dossier_test,data)




/ressources/My Drive/ressources/BDF/DATASET/train/1
/ressources/My Drive/ressources/BDF/DATASET/train/2
/ressources/My Drive/ressources/BDF/DATASET/train/3
/ressources/My Drive/ressources/BDF/DATASET/train/7
/ressources/My Drive/ressources/BDF/DATASET/train/6
/ressources/My Drive/ressources/BDF/DATASET/train/4
/ressources/My Drive/ressources/BDF/DATASET/train/5
/ressources/My Drive/ressources/BDF/DATASET/test/3
/ressources/My Drive/ressources/BDF/DATASET/test/7
/ressources/My Drive/ressources/BDF/DATASET/test/2
/ressources/My Drive/ressources/BDF/DATASET/test/4
/ressources/My Drive/ressources/BDF/DATASET/test/6
/ressources/My Drive/ressources/BDF/DATASET/test/1
/ressources/My Drive/ressources/BDF/DATASET/test/5


In [8]:
  def count_values(d):
    print(d)
    emotions={"1":"surprise", "2":"fear", "6":"angry", "7":"neutral", "5":"sad", "3":"disgust", "4":"happy"}
    count_value={}
    cont=0
    for key ,val in emotions.items():
        cont=0
        for value in d:
            if key== value[1]:
              cont=cont+1
        count_value[val]=cont
    return count_value



    return value_counts
count_values(data)

In [ ]:
#melange des données

random.shuffle(data)

In [ ]:
x=[]
y=[]

for features,label in data:
    x.append(features)
    y.append(label)

In [ ]:
X= np.array(x)
Y=np.array(y)

In [ ]:
X=X/255
X.shape
Y.shape

(15429,)

In [ ]:
model=Sequential()
model.add( Conv2D(64,(3,3),input_shape=X.shape[1:],activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add( Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add( Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Flatten())

model.add(Dense(7,activation='softmax'))

In [ ]:

model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
checkpoint=ModelCheckpoint(r'fer.keras',
                          monitor='val_accuracy',
                          mode='max',
                          save_best_only=True,
                          verbose=1)
earlystop=EarlyStopping(monitor='val_accuracy',
                        mode='max',
                       min_delta=0.001,
                       patience=20,
                       verbose=1,
                       restore_best_weights=True)

callbacks=[checkpoint,earlystop]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=42)

history = model.fit(
    X_train, Y_train,
    epochs=20,
    validation_data=(X_test, Y_test),
    callbacks=callbacks
)





Epoch 1/20
362/362 [==============================] - ETA: 0s - loss: 1.3965 - accuracy: 0.4927
Epoch 1: val_accuracy improved from -inf to 0.63867, saving model to fer.keras
362/362 [==============================] - 222s 610ms/step - loss: 1.3965 - accuracy: 0.4927 - val_loss: 1.0386 - val_accuracy: 0.6387
Epoch 2/20
362/362 [==============================] - ETA: 0s - loss: 0.9856 - accuracy: 0.6546
Epoch 2: val_accuracy improved from 0.63867 to 0.68092, saving model to fer.keras
362/362 [==============================] - 214s 590ms/step - loss: 0.9856 - accuracy: 0.6546 - val_loss: 0.8958 - val_accuracy: 0.6809
Epoch 3/20
362/362 [==============================] - ETA: 0s - loss: 0.8234 - accuracy: 0.7133
Epoch 3: val_accuracy improved from 0.68092 to 0.69181, saving model to fer.keras
362/362 [==============================] - 216s 597ms/step - loss: 0.8234 - accuracy: 0.7133 - val_loss: 0.8817 - val_accuracy: 0.6918
Epoch 4/20
362/362 [==============================] - ETA: 0s - 

In [ ]:


loss, accuracy = model.evaluate(X_train, Y_train)
print(f"Loss: {loss}, Accuracy: {accuracy}")

NameError: name 'model' is not defined

In [ ]:
from tensorflow.keras.models import load_model

model = load_model("fer.keras")
X_test[0].shape
"""
val=traiterImage("/c.jpeg")



image = np.expand_dims(val
, axis=0)  # Ajoute la dimnsion batch


predictions = model.predict(image)

print(f"{predictions}")

y_pred = model.predict(X_test)  # Prédictions sous forme de probabilités
y_pred_classes = np.argmax(y_pred, axis=1)
"""



from sklearn.metrics import confusion_matrix

# Calcul de la matrice de confusion
conf_matrix = confusion_matrix(Y_test, y_pred_classes)



"""
print("Matrice de confusion :\n", conf_matrix)
import matplotlib.pyplot as plt
import seaborn as sns

# Labels des classes (exemple pour 7 émotions)
class_names = ["Colère", "Dégoût", "Peur", "Joie", "Tristesse", "Surprise", "Neutre"]

# Visualisation
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Prédictions')
plt.ylabel('Vérités')
plt.title('Matrice de confusion')
plt.show()
"""







NameError: name 'y_pred_classes' is not defined

In [ ]:
# import dependencies
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time
import tensorflow as tf

# Importer les dépendances nécessaires
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import PIL.Image
import io

# Charger le modèle
model = load_model("fer.keras")  # Remplace "fer.keras" par le chemin de ton modèle
emotion_labels = ["Neutral", "Happy", "Sad", "Surprise", "Fear", "Angry", "Disgust"]

# Charger le cascade Haar pour la détection de visage
face_cascade = cv2.CascadeClassifier(cv2.samples.findFile(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'))

# Fonction pour convertir les données JS en image OpenCV
def js_to_image(js_data):
    img_data = b64decode(js_data.split(',')[1])
    img = PIL.Image.open(io.BytesIO(img_data))
    return np.array(img)

# Fonction pour dessiner des rectangles autour des visages détectés
def bbox_to_bytes(bbox_array):
    bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
    iobuf = io.BytesIO()
    bbox_PIL.save(iobuf, format='png')
    bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))
    return bbox_bytes

# JavaScript pour capturer la vidéo
def video_stream():
    js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;

    var pendingResolve = null;
    var shutdown = false;

    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
    }

    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);

      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "user"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);

      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;

      return stream;
    }
    async function stream_frame() {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      stream = await createDom();
      captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
      return captureCanvas.toDataURL('image/jpeg', 0.8);
    }
    ''')
    display(js)

# Fonction pour capturer un cadre
def video_frame():
    data = eval_js('stream_frame()')
    return data

# Lancer le flux vidéo
video_stream()

# Prédiction en temps réel
while True:
    js_reply = video_frame()
    if not js_reply:
        break

    # Convertir les données JS en image OpenCV
    img = js_to_image(js_reply)

    # Détection des visages (grayscale pour Haar Cascade)
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Superposition transparente
    bbox_array = np.zeros([480, 640, 4], dtype=np.uint8)

    for (x, y, w, h) in faces:
        # Extraire et prétraiter le visage en couleur
        face = img[y:y+h, x:x+w]  # Garder les 3 canaux couleur
        face = cv2.resize(face, (48, 48))  # Adapter à la taille du modèle
        face = face / 255.0  # Normaliser
        face = np.expand_dims(face, axis=0)  # Ajouter une dimension batch

        # Prédiction
        predictions = model.predict(face)
        emotion_index = np.argmax(predictions)
        emotion = emotion_labels[emotion_index]
        confidence = np.max(predictions) * 100

        # Dessiner le rectangle autour du visage
        cv2.rectangle(bbox_array, (x, y), (x+w, y+h), (255, 0, 0), 2)

        # Ajouter l'émotion prédite
        cv2.putText(
            bbox_array, f"{emotion} ({confidence:.1f}%)",
            (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2
        )

    # Ajouter un canal alpha pour la superposition
    bbox_array[:, :, 3] = (bbox_array.max(axis=2) > 0).astype(int) * 255

    # Convertir en base64 pour afficher l'image
    bbox = bbox_to_bytes(bbox_array)






In [ ]:
model.summary()